In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import scipy
import pandas as pd
import seaborn as sns

import cv2
import skvideo.io
import imageio
import pytube

import os
import time
import csv
import re
import humanize
from tqdm import tqdm

from skimage.measure import compare_mse, compare_ssim, compare_nrmse
from scipy.stats import wasserstein_distance
from scipy.spatial.distance import hamming
from imageai.Detection import ObjectDetection

Using TensorFlow backend.


In [3]:
def get_FPS(video_loc):
    """
    A function to compute the frame per second of the video
    
    Parameters
    ----------
    video_loc: str
        Location of the input video
        
    Returns
    -------
        A float number that is the FPS of the given video
    """
    
    video = cv2.VideoCapture(video_loc)
    fps = video.get(cv2.CAP_PROP_FPS)
    return fps

In [4]:
def add_mask(img, coordinates):
    """
    A function to add masks to the image on given coordinates
    
    Parameters
    ----------
    img: 
        Image object to add masks
    coordinates:
        Coordinates to add masks
        
    Returns
    -------
    image
        An image object with the masks   
    """
    
    mask = np.zeros(img.shape, dtype = 'uint8')
    
    for coord in coordinates:
        cv2.rectangle(mask, coord[0], coord[1], (255, 255, 255), -1);
        
    maskedImg = cv2.bitwise_and(img, mask)
    return maskedImg

In [5]:
def detect_object(img, model='resnet50_coco_best_v2.0.1.h5', minimum_prob=80):
    """
    A function to detect object from images using pre-trained network
    
    Parameters
    ----------
    img:
        Image object to detect objects
    model:
        The pre-trained network model (https://github.com/OlafenwaMoses/ImageAI/blob/master/imageai/Detection/README.md)
    minimum_prob: int
        Threshold to identify objects
    
    Returns
    -------   
        An array of dictionaries with the information of all detected objects
    """
    
    detector = ObjectDetection()
    detector.setModelTypeAsRetinaNet()
    # the pre-trained network should be in this folder
    detector.setModelPath(os.path.join('/home/idies/workspace/Storage/Cong/persistent/video', model))
    detector.loadModel()
    detections = detector.detectObjectsFromImage(input_image=img, input_type='array', minimum_percentage_probability=minimum_prob)
    return detections

In [6]:
def get_histogram(img):
    """
    A function to compute the histogram of an image
    """
    
    h, w = img.shape
    hist = [0.0] * 256
    for i in range(h):
        for j in range(w):
            hist[img[i, j]] += 1
    return np.array(hist) / (h * w)

In [7]:
def normalize_exposure(img):
    """
    A function to normalize the exposure of an image
    """
    
    img = img.astype(int)
    hist = get_histogram(img)
    # get the sum of vals accumulated by each position in hist
    cdf = np.array([sum(hist[:i+1]) for i in range(len(hist))])
    # determine the normalization values for each unit of the cdf
    sk = np.uint8(255 * cdf)
    # normalize each position in the output image
    h, w = img.shape
    normalized = np.zeros_like(img)
    for i in range(0, h):
        for j in range(0, w):
            normalized[i, j] = sk[img[i, j]]
    return normalized.astype(int)

In [8]:
def compare_emd(img1, img2):
    """
    A function to measure image similarity using EMD method
    """
    
    img1 = normalize_exposure(img1)
    img2 = normalize_exposure(img2)
    return wasserstein_distance(get_histogram(img1), get_histogram(img2))

In [9]:
def compare_orb(img1, img2, threshold):
    """
    A function to measure image similarity using ORB method
    """
    
    orb = cv2.ORB_create()
    kp_1, desc_1 = orb.detectAndCompute(img1, None)
    kp_2, desc_2 = orb.detectAndCompute(img2, None)
    if desc_1 is None or desc_2 is None:
        return 0
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(desc_1, desc_2)
    if len(matches) == 0:
        return 0
    similar = [i for i in matches if i.distance < threshold]
    return len(similar) / len(matches)

In [10]:
def dhash(img, hashSize=8):
    """
    A function to compute dHash of an image
    """
    
    # resize the input image, adding a single column (row) so we can compute the gradient
    resized_col = cv2.resize(img, (hashSize+1,hashSize))
    resized_row = cv2.resize(img, (hashSize,hashSize+1))
 
    # compute the (relative) gradient between adjacent pixels
    diff_col = resized_col[:,1:] > resized_col[:,:-1]
    diff_row = resized_row[1:,:] > resized_row[:-1,:]

    # convert the difference image to a hash
    return sum([2**i for (i, v) in enumerate(np.append(diff_col.flatten(), diff_row.flatten())) if v])

In [11]:
def compare_dhash(img1, img2, hashSize=8):
    """
    A function to measure image similarity using dHash method
    """
    
    h1 = [int(d) for d in str(int(dhash(img1)))]
    h2 = [int(d) for d in str(int(dhash(img2)))]
    
    if len(h1) == len(h2):
        dHash = hamming(h1, h2)
    else:
        dHash = 1
    return dHash

In [12]:
def compare_phash(img1, img2):
    """
    A function to measure image similarity using pHash method
    """
    
    phash = cv2.img_hash_PHash.create()
    return phash.compare(phash.compute(img1), phash.compute(img2))

In [13]:
def get_similar_image(image_df, method, ascending, idx, start, end, coords):
    """
    A function to find the similar image of a given image
    
    Parameters
    ----------
    image_df: 
        A dataframe with the location of images
    method:
        Methods used to measure image similarity, could be MSE, EMD, ORB, dHash, pHash
    ascending:
        Whether the metric used is ranked in ascending oreder
    idx:
        Target image index
    start:
        Start index of the image to search
    end:
        End index of the image to search
    coords:
        Coordinates to add mask
        
    Returns
    -------
        A dataframe with all the similar images (index/location)
    """
    
    results_dict = {}
            
    for i in range(start, min(end, image_df.shape[0])):
        
        img = imread(image_df['img_path'][idx])
        
        temp_img = imread(image_df['img_path'][i])
        
        if method == 'EMD' or method == 'ORB' or method == 'dHash':
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            temp_img = cv2.cvtColor(temp_img, cv2.COLOR_RGB2GRAY)
        
        img = add_mask(img, coords)
        temp_img = add_mask(temp_img, coords)
        
        detections = detect_object(img)
        carcoords = []
        for detection in detections:
            if detection.get('name') == 'car':
                temp = detection.get('box_points')
                carcoords.append(temp)
        
        tempdetections = detect_object(temp_img)
        for detection in tempdetections:
            if detection.get('name') == 'car':
                temp = detection.get('box_points')
                carcoords.append(temp)
        
        for carcoord in carcoords:
            cv2.rectangle(img,(carcoord[0],carcoord[1]),(carcoord[2],carcoord[3]),(0,0,0),-1);
            cv2.rectangle(temp_img,(carcoord[0],carcoord[1]),(carcoord[2],carcoord[3]),(0,0,0),-1);
        
        if method == 'MSE':
            results_dict[i] = [image_df['img_path'][i], image_df['img_index'][i], compare_mse(img, temp_img)]
        elif method == 'EMD':
            results_dict[i] = [image_df['img_path'][i], image_df['img_index'][i], compare_emd(img, temp_img)]
        elif method == 'ORB':
            results_dict[i] = [image_df['img_path'][i], image_df['img_index'][i], compare_orb(img, temp_img, threshold)]
        elif method == 'dHash':
            results_dict[i] = [image_df['img_path'][i], image_df['img_index'][i], compare_dhash(img, temp_img)]
        elif method == 'pHash':
            results_dict[i] = [image_df['img_path'][i], image_df['img_index'][i], compare_phash(img, temp_img)]
        else:
            results_dict[i] = [image_df['img_path'][i], image_df['img_index'][i], compare_ssim(img, temp_img, multichannel=True)]
    
    results_df = pd.DataFrame.from_dict(results_dict, orient='index')
    results_df.columns = ['img_path', 'img_index', method]
    results_df = results_df.sort_values(by=method, ascending=ascending)
    return results_df

In [15]:
data_path = '/home/idies/workspace/Storage/Cong/persistent/video/data'
result_path = '/home/idies/workspace/Storage/Cong/persistent/video/result'

In [ ]:
## Parameters for ORB method
threshold = 70

In [16]:
## Prior knowledge of the laptimes
prior_time = 60

## FPS, could be computed using function `get_FPS`
fps = 29.93930396046777

## Dataframe from function `video_to_image2` in video2image.ipynb
df = pd.read_csv(os.path.join(data_path, 'train091108_raw.csv'))

## Parameters for the function `get_similar_image`
initial_idx = 0
start = initial_idx + np.round(fps*prior_time).astype(int)
end = 3600

method = 'pHash'
ascending = True

coords = [[(0, 600), (1920, 740)], [(300, 0), (1920, 740)]]

In [ ]:
start_time = time.time()
similar_imgs = get_similar_image(df, method, ascending, initial_idx, start, end, coords)
elapsedTime = time.time() - start_time

In [9]:
def weighted_index(df, n, method, ascending):
    """
    A function to get the weighted index of the similar images
    """
    
    temp = df.copy()[:n]
    if ascending:
        temp['weight'] = (1/(temp[method]+1e-8)) / sum(1/(temp[method]+1e-8))
    else:
        temp['weight'] = temp[method] / sum(temp[method])
    return sum(temp['img_index']*temp['weight'])

In [58]:
def lap_time2(initial_idx, end, fps, topn, df, method, ascending, coords):
    """
    A function to estimate the laptimes
    
    Most parameters are for function `get_similar_image`
    """
    
    results_dict = {}
    start = initial_idx + np.round(fps*prior_time).astype(int)
    
    #img_idxs = np.arange(initial_idx, initial_idx+3).tolist()
    #img_idxs = np.arange(initial_idx+3, initial_idx+6).tolist()
    #img_idxs = np.arange(initial_idx+6, initial_idx+9).tolist()
    #img_idxs = np.arange(initial_idx+9, initial_idx+12).tolist()
    #img_idxs = np.arange(initial_idx+12, initial_idx+15).tolist()
    #img_idxs = np.arange(initial_idx+15, initial_idx+18).tolist()
    #img_idxs = np.arange(initial_idx+18, initial_idx+21).tolist()
    #img_idxs = np.arange(initial_idx+21, initial_idx+24).tolist()
    #img_idxs = np.arange(initial_idx+24, initial_idx+27).tolist()
    #img_idxs = np.arange(initial_idx+27, np.round(initial_idx+fps).astype(int)).tolist()
    
    img_idxs = np.arange(initial_idx, initial_idx+5).tolist()
    #img_idxs = np.arange(initial_idx+5, initial_idx+10).tolist()
    #img_idxs = np.arange(initial_idx+10, initial_idx+15).tolist()
    #img_idxs = np.arange(initial_idx+15, initial_idx+20).tolist()
    #img_idxs = np.arange(initial_idx+20, initial_idx+25).tolist()
    #img_idxs = np.arange(initial_idx+25, np.round(initial_idx+fps).astype(int)).tolist()
    
    #img_idxs = np.arange(initial_idx, initial_idx+10).tolist()
    #img_idxs = np.arange(initial_idx+10, initial_idx+20).tolist()
    #img_idxs = np.arange(initial_idx+20, np.round(initial_idx+fps).astype(int)).tolist()
    
    #img_idxs = np.choice(np.arange(initial_idx, np.round(initial_idx+fps).astype(int)), 10).tolist()
    
    for idx in img_idxs:
        #img = imread(df['img_path'][idx])
        start_time = time.time()
        similar_imgs = get_similar_image(df, method, ascending, idx, start, end, coords)
        elapsedTime = time.time() - start_time
        results_dict[idx] = [df['img_path'][idx], df['img_index'][idx], 
                             (np.mean(similar_imgs.img_index[similar_imgs[method]==(similar_imgs[method].iloc[0])])-df['img_index'][idx])/fps, 
                             (weighted_index(similar_imgs, topn, method, ascending)-df['img_index'][idx])/fps, 
                             elapsedTime, method]
    
    results_df = pd.DataFrame.from_dict(results_dict, orient='index')
    results_df.columns = ['img_path', 'img_index', 'lap_time', 'weighted_lap_time', 'elapsedTime', 'method']
    return results_df

In [6]:
## An example
df = pd.read_csv(os.path.join(data_path, 'train091108_raw.csv'))

initial_idx = 21681
end = 26146
prior_time = 60

fps = 29.93930396046777
topn = 5

method = 'pHash'
ascending = True

coords = [[(0, 600), (1920, 740)], [(300, 0), (1920, 740)]]

In [ ]:
test = lap_time2(initial_idx, end, fps, topn, df, method, ascending, coords)
test['lap'] = 9
test.to_csv(os.path.join(result_path, '091108', 'lap9_pHash_6.csv'), index=False)

In [52]:
temp1 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_1.csv'))
temp2 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_2.csv'))
temp3 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_3.csv'))
temp4 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_4.csv'))
temp5 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_5.csv'))
temp6 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_6.csv'))
# temp7 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_7.csv'))
# temp8 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_8.csv'))
# temp9 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_9.csv'))
# temp10 = pd.read_csv(os.path.join(result_path, '091108', 'lap9_pHash_10.csv'))
temp = temp1.append(temp2)
temp = temp.append(temp3)
temp = temp.append(temp4)
temp = temp.append(temp5)
temp = temp.append(temp6)
# temp = temp.append(temp7)
# temp = temp.append(temp8)
# temp = temp.append(temp9)
# temp = temp.append(temp10)

# initial_idx
print(int(temp['lap_time'].iloc[0] * fps) + initial_idx)

# start
print(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int))

# end
print(int(int(temp['lap_time'].iloc[0] * fps) + initial_idx + np.round(fps*prior_time).astype(int) + np.mean(temp['lap_time']) * fps))

24359
26155
28828


In [53]:
temp.to_csv(os.path.join(result_path, '091108', 'lap9_pHash.csv'), index=False)

In [54]:
temp = pd.read_csv('./result/091108/lap9_pHash.csv')
np.median(temp['lap_time'])

89.54227315615418